In [1]:
import cv2
import numpy as np
import os

In [2]:
def show_images(imgList):
    while True:
        for (name,img) in imgList:
            cv2.imshow(name,img)
        pressed_key = cv2.waitKey(0)
        if pressed_key==27:
            break
    cv2.destroyAllWindows()

In [3]:
def jacob_warp_affine(x,y):
    return np.array([[x,0,y,0,1,0],[0,x,0,y,0,1]],dtype=np.float32) ##(2,3)

In [4]:
def linearize_warp(wf):
    assert (wf.shape==(2,3))
    return np.float32([[wf[0,0]-1.0],[wf[1,0]],[wf[0,1]],[wf[1,1]-1.0],[wf[0,2]],[wf[1,2]]]) ##(6,1)

In [5]:
def make_warp(p):
    assert (p.shape==(6,1))
    return np.float32([[p[0,0]+1.0,p[2,0],p[4,0]],[p[1,0],p[3,0]+1.0,p[5,0]]]) ##(2,3)

In [22]:
def inverse_param(p):
    assert (p.shape==(6,1))
    denom = ((1+p[0,0])*(1+p[3,0]) - (p[1,0]*p[2,0]))
    invp = np.zeros(p.shape,dtype=np.float32)
    invp[0,0] = (-p[0,0])-(p[0,0]*p[3,0])+(p[1,0]*p[2,0])
    invp[1,0] = (-p[1,0])
    invp[2,0] = (-p[2,0])
    invp[3,0] = (-p[3,0])-(p[0,0]*p[3,0])+(p[1,0]*p[2,0])
    invp[4,0] = (-p[4,0])-(p[3,0]*p[4,0])+(p[2,0]*p[5,0])
    invp[5,0] = (-p[5,0])-(p[0,0]*p[5,0])+(p[1,0]*p[4,0])
    invp = invp/denom
    return invp ##(6,1)

In [23]:
def compose_param(p1,p2):
    assert (p1.shape==(6,1))
    assert (p2.shape==(6,1))
    comp = np.zeros(p1.shape,dtype=np.float32)
    comp[0,0] = p1[0,0]+p2[0,0]+(p1[0,0]*p2[0,0])+(p1[2,0]*p2[1,0])
    comp[1,0] = p1[1,0]+p2[1,0]+(p1[1,0]*p2[0,0])+(p1[3,0]*p2[1,0])
    comp[2,0] = p1[2,0]+p2[2,0]+(p1[0,0]*p2[2,0])+(p1[2,0]*p2[3,0])
    comp[3,0] = p1[3,0]+p2[3,0]+(p1[1,0]*p2[2,0])+(p1[3,0]*p2[3,0])
    comp[4,0] = p1[4,0]+p2[4,0]+(p1[0,0]*p2[4,0])+(p1[2,0]*p2[5,0])
    comp[5,0] = p1[5,0]+p2[5,0]+(p1[1,0]*p2[4,0])+(p1[3,0]*p2[5,0])
    return comp ##(6,1)

In [95]:
def gaussNewton(img,template,warp,epsilon,transform):
    template = template.astype('float32')
    img = img.astype('float32')
    assert (len(img.shape)==2), "Grayscale image should be given"
    ## template is the target image
    ## img is initial image
    ## warp is the initial guess warp ([1+p1,p3,p5],[p2,1+p4,p6])
    ## transform is one of "AFFINE" "TRANSLATIONAL"
    ## epsilon defines the ending condition
    

    template_shape = template.shape
    tmplx,tmply = template_shape
    
    ## Calculating gradients (3)
    
    # kerX = np.float32([[-0.5,0.0,0.5]])
    # kerY = np.float32([-0.5,0.0,0.5])
    # gradX = cv2.filter2D(template,-1,kerX)
    # gradY = cv2.filter2D(template,-1,kerY)
    
    gradTx = cv2.Sobel(template,cv2.CV_64F,1,0,ksize=3) ##(x,y)
    gradTy = cv2.Sobel(template,cv2.CV_64F,0,1,ksize=3) ##(x,y)
    grad = np.stack([gradTx,gradTy],axis=-1) ##(x,y,2)
    grad = np.expand_dims(grad,axis=2) ##(x,y,1,2)
    
    ## Calculating jacobian of template image (4)
    jacob_tmpl = np.zeros([tmplx,tmply,2,6]) ## (x,y,2,6)
    for i in range(tmplx):
        for j in range(tmply):
            # because i,j in numpy image correspond to j,i in image axis
            jacob_tmpl[i,j] = jacob_warp_affine(j,i)
    
    ## Calculating steepest descent (5)
    steep_desc = np.matmul(grad,jacob_tmpl) ##(x,y,1,6)
    steep_desc_trans = np.transpose(steep_desc,[0,1,3,2]) ##(x,y,6,1)
    
    
    ## Calculating Hessian matrix (6)
    hess = np.sum(np.sum(np.matmul(steep_desc_trans,steep_desc),axis=0),axis=0) ##(6,6)
    inv_hess = np.linalg.inv(hess) ##(6,6)
    
    delP = np.ones([6,1],dtype=np.float32)
    iterations = 0
#     while(np.linalg.norm(delP,2) > epsilon): #2-Norm end condition
    while(np.linalg.norm(delP) > epsilon): #Frobenius norm end condition
        
        ## Calculation warp of given image with current guess (1)
        warp_img = cv2.warpAffine(img,warp,(tmply,tmplx)) ##(x,y)
        ## Calculate error image (2)
        err_img = warp_img - template ##(x,y)
        err_img = np.expand_dims(np.expand_dims(err_img,-1),-1) ##(x,y,1,1)
        
        ## Computer other term (7)
        other_term = np.sum(np.sum(np.matmul(steep_desc_trans,err_img),axis=0),axis=0) ##(6,1)
        ## Computing delP (8)
        delP = np.matmul(inv_hess,other_term) ##(6,1)
        
        ## Updating warp (9)
        initP = linearize_warp(warp) ##(6,1)
#         invP = inverse_param(delP) ##(6,1)
        invP = delP
        nextP = compose_param(initP,invP) ##(6,1)
        warp = make_warp(nextP) ##(2,3)
        iterations += 1
        
#         print("Iteration %d ; Norm %f" %(iterations,np.linalg.norm(delP,ord='fro')))
        
        if iterations > 5000:
            break
    return warp
    

In [96]:
def lukasKanadeTracker(img1,img2,initialwarp,epsilon, transform='AFFINE'):
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    return gaussNewton(img1,img2,initialwarp,epsilon,transform)

## Example

In [17]:
# img1 = cv2.imread("./images/image1.jpg")
# img2 = cv2.imread("./images/image2.jpg")

In [32]:
img1 = cv2.imread("./images/room_left.jpg")
img2 = cv2.imread("./images/room_right.jpg")

In [19]:
# img = cv2.imread("./images/room_left.jpg")
# img1 = img[50:350,50:450]
# img2 = img[55:355,55:455]

In [26]:
show_images([("img1",img1),("img2",img2)])

In [ ]:
r = cv2.selectROI("Select window of interest",img=img1,fromCenter=False,showCrossair=False)
cv2.destroyAllWindows()
tmpl = img1[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
initial_warp = np.float32([[1.0,0.0,-float(r[0])],[0.0,1.0,-float(r[1])]])

final_warp = lukasKanadeTracker( img2, tmpl, initial_warp, 0.0010, transform='AFFINE')
final_warp[0,2] -= initial_warp[0,2]
final_warp[1,2] -= initial_warp[1,2]

In [33]:
initial_warp = np.float32([[1.0,0.0,0],[0.0,1.0,0]])
final_warp = lukasKanadeTracker( img2, img1, initial_warp, 0.0010, transform='AFFINE')

Iteration 1 ; Norm 0.025873
Iteration 2 ; Norm 0.026199
Iteration 3 ; Norm 0.026484
Iteration 4 ; Norm 0.026835
Iteration 5 ; Norm 0.027156
Iteration 6 ; Norm 0.027445
Iteration 7 ; Norm 0.027718
Iteration 8 ; Norm 0.028051
Iteration 9 ; Norm 0.028308
Iteration 10 ; Norm 0.028526
Iteration 11 ; Norm 0.028756
Iteration 12 ; Norm 0.029091
Iteration 13 ; Norm 0.029432
Iteration 14 ; Norm 0.029843
Iteration 15 ; Norm 0.030306
Iteration 16 ; Norm 0.030760
Iteration 17 ; Norm 0.031375
Iteration 18 ; Norm 0.032066
Iteration 19 ; Norm 0.032856
Iteration 20 ; Norm 0.033561
Iteration 21 ; Norm 0.034393
Iteration 22 ; Norm 0.035260
Iteration 23 ; Norm 0.036199
Iteration 24 ; Norm 0.037097
Iteration 25 ; Norm 0.037927
Iteration 26 ; Norm 0.038756
Iteration 27 ; Norm 0.039634
Iteration 28 ; Norm 0.040525
Iteration 29 ; Norm 0.041433
Iteration 30 ; Norm 0.042308
Iteration 31 ; Norm 0.043123
Iteration 32 ; Norm 0.044016
Iteration 33 ; Norm 0.044860
Iteration 34 ; Norm 0.045717
Iteration 35 ; Norm 0.0

Iteration 284 ; Norm 0.064162
Iteration 285 ; Norm 0.063942
Iteration 286 ; Norm 0.063703
Iteration 287 ; Norm 0.063488
Iteration 288 ; Norm 0.063338
Iteration 289 ; Norm 0.063365
Iteration 290 ; Norm 0.063259
Iteration 291 ; Norm 0.063277
Iteration 292 ; Norm 0.063419
Iteration 293 ; Norm 0.063519
Iteration 294 ; Norm 0.063762
Iteration 295 ; Norm 0.064072
Iteration 296 ; Norm 0.064431
Iteration 297 ; Norm 0.064925
Iteration 298 ; Norm 0.065489
Iteration 299 ; Norm 0.066127
Iteration 300 ; Norm 0.066931
Iteration 301 ; Norm 0.067833
Iteration 302 ; Norm 0.068824
Iteration 303 ; Norm 0.069940
Iteration 304 ; Norm 0.071242
Iteration 305 ; Norm 0.072728
Iteration 306 ; Norm 0.074246
Iteration 307 ; Norm 0.075896
Iteration 308 ; Norm 0.077492
Iteration 309 ; Norm 0.079202
Iteration 310 ; Norm 0.080877
Iteration 311 ; Norm 0.082576
Iteration 312 ; Norm 0.084247
Iteration 313 ; Norm 0.085933
Iteration 314 ; Norm 0.087514
Iteration 315 ; Norm 0.089075
Iteration 316 ; Norm 0.090556
Iteration 

Iteration 564 ; Norm 0.027647
Iteration 565 ; Norm 0.026913
Iteration 566 ; Norm 0.026235
Iteration 567 ; Norm 0.025542
Iteration 568 ; Norm 0.024890
Iteration 569 ; Norm 0.024260
Iteration 570 ; Norm 0.023659
Iteration 571 ; Norm 0.023063
Iteration 572 ; Norm 0.022578
Iteration 573 ; Norm 0.022002
Iteration 574 ; Norm 0.021551
Iteration 575 ; Norm 0.021085
Iteration 576 ; Norm 0.020638
Iteration 577 ; Norm 0.020321
Iteration 578 ; Norm 0.019986
Iteration 579 ; Norm 0.019664
Iteration 580 ; Norm 0.019307
Iteration 581 ; Norm 0.019056
Iteration 582 ; Norm 0.018834
Iteration 583 ; Norm 0.018582
Iteration 584 ; Norm 0.018368
Iteration 585 ; Norm 0.018142
Iteration 586 ; Norm 0.017937
Iteration 587 ; Norm 0.017744
Iteration 588 ; Norm 0.017588
Iteration 589 ; Norm 0.017417
Iteration 590 ; Norm 0.017217
Iteration 591 ; Norm 0.017021
Iteration 592 ; Norm 0.016840
Iteration 593 ; Norm 0.016696
Iteration 594 ; Norm 0.016502
Iteration 595 ; Norm 0.016347
Iteration 596 ; Norm 0.016160
Iteration 

Iteration 846 ; Norm 0.004121
Iteration 847 ; Norm 0.004074
Iteration 848 ; Norm 0.004064
Iteration 849 ; Norm 0.004046
Iteration 850 ; Norm 0.004040
Iteration 851 ; Norm 0.004050
Iteration 852 ; Norm 0.004029
Iteration 853 ; Norm 0.004028
Iteration 854 ; Norm 0.004030
Iteration 855 ; Norm 0.004000
Iteration 856 ; Norm 0.003994
Iteration 857 ; Norm 0.003964
Iteration 858 ; Norm 0.003928
Iteration 859 ; Norm 0.003887
Iteration 860 ; Norm 0.003866
Iteration 861 ; Norm 0.003837
Iteration 862 ; Norm 0.003817
Iteration 863 ; Norm 0.003789
Iteration 864 ; Norm 0.003789
Iteration 865 ; Norm 0.003771
Iteration 866 ; Norm 0.003755
Iteration 867 ; Norm 0.003745
Iteration 868 ; Norm 0.003752
Iteration 869 ; Norm 0.003742
Iteration 870 ; Norm 0.003723
Iteration 871 ; Norm 0.003715
Iteration 872 ; Norm 0.003688
Iteration 873 ; Norm 0.003665
Iteration 874 ; Norm 0.003604
Iteration 875 ; Norm 0.003593
Iteration 876 ; Norm 0.003560
Iteration 877 ; Norm 0.003549
Iteration 878 ; Norm 0.003526
Iteration 

Iteration 1131 ; Norm 0.001468
Iteration 1132 ; Norm 0.001465
Iteration 1133 ; Norm 0.001458
Iteration 1134 ; Norm 0.001465
Iteration 1135 ; Norm 0.001451
Iteration 1136 ; Norm 0.001450
Iteration 1137 ; Norm 0.001442
Iteration 1138 ; Norm 0.001427
Iteration 1139 ; Norm 0.001434
Iteration 1140 ; Norm 0.001433
Iteration 1141 ; Norm 0.001428
Iteration 1142 ; Norm 0.001427
Iteration 1143 ; Norm 0.001418
Iteration 1144 ; Norm 0.001410
Iteration 1145 ; Norm 0.001423
Iteration 1146 ; Norm 0.001404
Iteration 1147 ; Norm 0.001409
Iteration 1148 ; Norm 0.001399
Iteration 1149 ; Norm 0.001392
Iteration 1150 ; Norm 0.001395
Iteration 1151 ; Norm 0.001399
Iteration 1152 ; Norm 0.001389
Iteration 1153 ; Norm 0.001394
Iteration 1154 ; Norm 0.001374
Iteration 1155 ; Norm 0.001375
Iteration 1156 ; Norm 0.001359
Iteration 1157 ; Norm 0.001374
Iteration 1158 ; Norm 0.001385
Iteration 1159 ; Norm 0.001373
Iteration 1160 ; Norm 0.001351
Iteration 1161 ; Norm 0.001351
Iteration 1162 ; Norm 0.001364
Iteratio

In [34]:
warped_img = cv2.warpAffine(img2,final_warp,(img1.shape[1],img1.shape[0]))

In [35]:
show_images([("img1",img1),("img2",img2),("warp",warped_img)])

In [ ]:
warp2 = warped_img.copy()
warp3 = warped_img.copy()

In [ ]:
warp2[warp2==0] = img1[warp2==0]
warp3[warp3==0] = img2[warp3==0]

In [ ]:
cv2.selectROI("Select window of interest",img=img1,fromCenter=False,showCrossair=False)

In [ ]:
sho

In [ ]:
cv2.destroyAllWindows()

In [ ]:
show_images([("wp1",warp2),("wp2",warp3)])

## Test video

In [110]:
video_file = "./videos/shaky_book.mov"
result_path = "./results/video/shaky_book/"
os.makedirs(result_path,exist_ok=True)
os.makedirs(result_path+"stable",exist_ok=True)
os.makedirs(result_path+"unstable",exist_ok=True)

In [80]:
cols,rows = (400,600)

In [97]:
cap = cv2.VideoCapture(video_file)
ret,frame=  cap.read()
resz_frame = cv2.resize(frame,(rows,cols))
r = cv2.selectROI("Select window of interest",img=resz_frame,fromCenter=False,showCrossair=False)
cv2.destroyAllWindows()
tmpl = resz_frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
initial_warp = np.float32([[1.0,0.0,-float(r[0])],[0.0,1.0,-float(r[1])]])
warp = initial_warp.copy()

In [98]:
stable_frames = []
unstable_frames = []

In [99]:
show_images([("tmpl",tmpl)])

In [100]:
cnt = 0
while True:
    ret,frame=  cap.read()
    if not ret:
        break
    resz_frame = cv2.resize(frame,(rows,cols))
    unstable_frames.append(resz_frame)
    print(cnt)
    print("Starting")
    warp = lukasKanadeTracker( resz_frame, tmpl, warp, 0.0010, transform='AFFINE')
    frame_warp = warp.copy()
    frame_warp[0,2] -= initial_warp[0,2]
    frame_warp[1,2] -= initial_warp[1,2]
    warped_img = cv2.warpAffine(resz_frame,frame_warp,(rows,cols))
    stable_frames.append(warped_img)
    print("Done")
    cnt+=1
#     if cnt>5000:
#         break
#     cv2.imshow("video",resz_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

0
Starting
Done
1
Starting
Done
2
Starting
Done
3
Starting
Done
4
Starting
Done
5
Starting
Done
6
Starting
Done
7
Starting
Done
8
Starting
Done
9
Starting
Done
10
Starting
Done
11
Starting
Done
12
Starting
Done
13
Starting
Done
14
Starting
Done
15
Starting
Done
16
Starting
Done
17
Starting
Done
18
Starting
Done
19
Starting
Done
20
Starting
Done
21
Starting
Done
22
Starting
Done
23
Starting
Done
24
Starting
Done
25
Starting
Done
26
Starting
Done
27
Starting
Done
28
Starting
Done
29
Starting
Done
30
Starting
Done
31
Starting
Done
32
Starting
Done
33
Starting
Done
34
Starting
Done
35
Starting
Done
36
Starting
Done
37
Starting
Done
38
Starting
Done
39
Starting
Done
40
Starting
Done
41
Starting
Done
42
Starting
Done
43
Starting
Done
44
Starting
Done
45
Starting
Done
46
Starting
Done
47
Starting
Done
48
Starting
Done
49
Starting
Done
50
Starting
Done
51
Starting
Done
52
Starting
Done
53
Starting
Done
54
Starting
Done
55
Starting
Done
56
Starting
Done
57
Starting
Done
58
Starting
Done
59
Star

KeyboardInterrupt: 

In [112]:
for i in range(len(stable_frames)):
    cv2.imshow("stable",stable_frames[i])
    path = result_path+"stable/"+('{:04}'.format(i))+".jpg"
    print(path)
    cv2.imwrite(path,stable_frames[i])
    cv2.imshow("unstable",unstable_frames[i])
    cv2.imwrite(result_path+"unstable/"+('{:04}'.format(i))+".jpg",unstable_frames[i])
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

./results/video/shaky_book/stable/0000.jpg
./results/video/shaky_book/stable/0001.jpg
./results/video/shaky_book/stable/0002.jpg
./results/video/shaky_book/stable/0003.jpg
./results/video/shaky_book/stable/0004.jpg
./results/video/shaky_book/stable/0005.jpg
./results/video/shaky_book/stable/0006.jpg
./results/video/shaky_book/stable/0007.jpg
./results/video/shaky_book/stable/0008.jpg
./results/video/shaky_book/stable/0009.jpg
./results/video/shaky_book/stable/0010.jpg
./results/video/shaky_book/stable/0011.jpg
./results/video/shaky_book/stable/0012.jpg
./results/video/shaky_book/stable/0013.jpg
./results/video/shaky_book/stable/0014.jpg
./results/video/shaky_book/stable/0015.jpg
./results/video/shaky_book/stable/0016.jpg
./results/video/shaky_book/stable/0017.jpg
./results/video/shaky_book/stable/0018.jpg
./results/video/shaky_book/stable/0019.jpg
./results/video/shaky_book/stable/0020.jpg
./results/video/shaky_book/stable/0021.jpg
./results/video/shaky_book/stable/0022.jpg
./results/v

In [113]:
img = unstable_frames[144]

array([[  3.40521336e-04,  -1.33204412e-05,   7.83020172e+01],
       [ -1.39635185e-05,   1.19805336e-05,   4.43701134e+01]], dtype=float32)

In [118]:
show_images([("im1",tmpl),("im3",unstable_frames[144])])

In [129]:
warped_img = cv2.warpAffine(img,warp,(tmpl.shape[1],tmpl.shape[0]))

In [131]:
show_images([("im1",tmpl),("im3",img)])

In [127]:
warp = lukasKanadeTracker( img, tmpl, initial_warp, 0.0010, transform='AFFINE')

In [128]:
warp

array([[  9.47210193e-01,  -4.54894751e-02,  -5.62678108e+01],
       [ -2.29301959e-01,   1.99989498e-01,   4.92323265e+01]], dtype=float32)